In [60]:
import json
import os
import time
import requests

from operator import itemgetter


# Let's overlay EV charging stations, their reviews, and relevant traffic flows

### Google maps API

In [65]:
def create_reviews_data(api_key, place_json):
    place_id = place_json.get("place_id")
    if place_id:
        rating_and_reviews_url = f"https://maps.googleapis.com/maps/api/place/details/json?placeid={place_id}&key={api_key}"
        response = requests.get(rating_and_reviews_url)
        reviews = response.json().get("result").get("reviews")
        if reviews:
            anon_reviews = [{k: review[k] for k in ["rating", "relative_time_description", "text", "time"]} for review in reviews]
            anon_reviews.sort(key=itemgetter("time"), reverse=True)
            place_json["reviews"] = anon_reviews
    return place_json



In [30]:
def get_coordinates_of_evs(api_key, coordinate, radius, keywords):
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={coordinate}&radius={radius}&keyword={keywords}&key={api_key}"
    final_data = []
    while True:
        response = requests.get(url)
        jj = response.json()
        results = jj.get("results")
        for result in results:          
            name = result['name']          
            place_id = result ['place_id']          
            lat = result['geometry']['location']['lat']          
            lng = result['geometry']['location']['lng']             
            rating = result['rating']          
            types = result['types']          
            vicinity = result['vicinity']          
            business_status = result['business_status']            
            data = {"name": name, "place_id": place_id, "lat": lat, "long": lng, "rating": rating, "types": types, "vicinity": vicinity, "business_status": business_status}          
            final_data.append(data)           

            time.sleep(1)

        if "next_page_token" not in jj:
            break
        else:
            url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken={jj['next_page_token']}&key={api_key}" 
    return final_data
        



In [10]:
cle_radius = 160934/2
coordinate = "41.4870125,-81.7216216"
keyword="ev charger"
#api_key = os.getenv("MAPS_API_KEY")

In [44]:
cle_ev_chargers = get_coordinates_of_evs(api_key, coordinate, cle_radius, keyword)

In [66]:
cle_ev_charger_reviewed = []
for charger_place in cle_ev_chargers:
    reviewed = create_reviews_data(api_key, charger_place)
    cle_ev_charger_reviewed.append(reviewed)
   


In [68]:
len(cle_ev_charger_reviewed)

60

# Story Map (EDA)

In [79]:
import gmaps

In [80]:
gmaps.configure(api_key=api_key)

In [95]:
fig = gmaps.figure(map_type="ROADMAP", center=(41.4870125,-81.7216216), zoom_level=13)

In [110]:
import collections
collections.Iterable = collections.abc.Iterable
collections.Sequence = collections.abc.Sequence

In [126]:
locations = [(x["lat"], x["long"]) for x in cle_ev_charger_reviewed]
info_box_template = """<dl><dt>Review</dt><dd>{rating}</dd><dt>Text</dt><dd>{text}</dd></d"""
ev_info = info_box_template.format(**review) for review in reviews 
ev_layer= gmaps.symbol_layer(locations, info_box_content=[json.dumps(x.get("reviews", []), indent=4) for x in cle_ev_charger_reviewed])
fig.add_layer(ev_layer)


In [127]:
traffic_layer= gmaps.traffic_layer()
fig.add_layer(traffic_layer)


In [128]:
fig

Figure(layout=FigureLayout(height='420px'))